In [1]:
import sys
!{ sys . executable } -m pip install pandas
import pandas as pd
import numpy as np 
pd . read_excel("/Users/ma.mercedesgarciafagalde/Desktop/Base de Datos - Junio 11.xlsx")
df = pd . read_excel("/Users/ma.mercedesgarciafagalde/Desktop/Base de Datos - Junio 11.xlsx")
print(df) 


     Year  Country  BN.CAB.XOKA.GD.ZS  BX.KLT.DINV.WD.GD.ZS  DBT-EXP  Default  \
0    1970        1                NaN                   NaN      3.0        0   
1    1971        1                NaN                   NaN      6.0        0   
2    1972        1                NaN                   0.0      2.0        0   
3    1973        1                NaN                   0.0      3.0        0   
4    1974        1                NaN                   0.0      6.0        0   
..    ...      ...                ...                   ...      ...      ...   
290  2024        5                NaN                   NaN      NaN        0   
291  2025        5                NaN                   NaN      NaN        0   
292  2026        5                NaN                   NaN      NaN        0   
293  2027        5                NaN                   NaN      NaN        0   
294  2028        5                NaN                   NaN      NaN        0   

     DT.DIS.DECB.CD  DT.DIS

In [2]:
## Reemplazamos los valores de variables con missing values por la media de cada columna
df = df.fillna(df.mean())
print(df)


     Year  Country  BN.CAB.XOKA.GD.ZS  BX.KLT.DINV.WD.GD.ZS   DBT-EXP  \
0    1970        1          -0.813333              1.514286  3.000000   
1    1971        1          -0.813333              1.514286  6.000000   
2    1972        1          -0.813333              0.000000  2.000000   
3    1973        1          -0.813333              0.000000  3.000000   
4    1974        1          -0.813333              0.000000  6.000000   
..    ...      ...                ...                   ...       ...   
290  2024        5          -0.813333              1.514286  3.360784   
291  2025        5          -0.813333              1.514286  3.360784   
292  2026        5          -0.813333              1.514286  3.360784   
293  2027        5          -0.813333              1.514286  3.360784   
294  2028        5          -0.813333              1.514286  3.360784   

     Default  DT.DIS.DECB.CD  DT.DIS.DEGG.CD  DT.DIS.DEPS.CD  DT.DIS.DLTF.CD  \
0          0    2.850000e+05    2.230567e+0

In [3]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt


def evalua_metodo(modelo, X_train, Y_train, X_test, Y_test):
    '''
    Esta función calcula la matriz de confusion, el accuracy y el AUC, ademas de graficar la curva roc y calcular el ECM. 
    Al final la función nos devuelve un data frame con los resultados de estas métricas para el modelo seleccionado en 
    base a los datos ingresados.
    Input: 
        modelo: un modelo con los parámetros ya definidos.
        X_train, Y_train, X_test, Y_test: Datos de entrenamiento y test ya divididos.
    Output:
        Matriz especificando VP, FP, VN, FN, Accuracy, AUC, ECM.
        Además imprime la matriz de confusión, el Accuracy, el AUC y grafica la curva de ROC.
    '''
    # Ajustamos el modelo
    modelo.fit(X_train, Y_train)
    
    # Realizamos predicción sobre base test
    global Y_pred 
    Y_pred = modelo.predict(X_test)
    
    # Calculamos el accuracy y matriz de confusion
    matriz_confusion = confusion_matrix(Y_test, Y_pred)
    accuracy = accuracy_score(Y_test, Y_pred)
    auc = roc_auc_score(Y_test, Y_pred)
    

    # Graficamos la curva de roc
    fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)
    
    #Calculamos ECM
    ecm = mean_squared_error(Y_pred, Y_test)
    
    #Guardamos los resultados en un dataframe creado con más columnas que nos servirán para las otras funciones
    output = pd.DataFrame(columns = ["Modelo", "Penalty", "Hiperparámetro", "VP", "FP", "VN", "FN", "Accuracy", "AUC", "ECM"])
    output = output.append({"VP" : matriz_confusion[0][0], "FP": matriz_confusion[0][1], "VN": matriz_confusion[1][1], "FN": matriz_confusion[1][0], "Accuracy" : accuracy, "AUC" : auc, "ECM" : ecm}, ignore_index = True)

    return output, fpr, tpr

In [4]:
# LISTA EN VEZ DE DICCIONARIO

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import boxplot

def cross_validation(X, y, nombre, modelo, penalty = "l1", hyper_list = [1, 5, 10], p = 5):
    '''
    Esta función realiza p-fold cross validation para distintos modelos, lista de hiperparámetros y data set determinado. Para eso, itera sobre una lista de hiperparámetros, crea p distintas particiones de la data y guarda los resultados en un dataframe.
    Input: 
        X: Matriz de predictores.
        y: Variable dependiente.
        modelo: Modelo a utilizar.
        penalty: Tipo de regularización ("l1": Lasso, "l2": Ridge, "elasticnet": Elastic Net).
        hyper_list: Lista de hiperparámetros a evaluar.
        p: Cantidad de particiones a realizar.
    Output:
        Dataframe por modelo e hierparámetro con el ECM promedio de las distintas particiones.
    '''
    
    #Creamos la matriz en la que vamos a guardar los outputs
    datos = pd.DataFrame(columns=["Modelo", "Particion", "Penalty", "Lambda", "ECM"])
    
    #Reseteamos el indice de las matrices a utilizar para que funcione bien el split
    X.reset_index(inplace = True, drop = True)
    y.reset_index(inplace = True, drop = True)
    
    
    #Iteramos sobre los lambdas
    for hiper in hyper_list:
        
        kf = KFold(n_splits=p, shuffle=True, random_state = 1265465)
        #Iteramos sobre las distintas particiones
        for i, (train_index, test_index) in enumerate(kf.split(X)): 
            
            #Separamos los datos en train y test
            x_train, x_test = X.reindex(train_index), X.reindex(test_index)
            y_train, y_test = y.reindex(train_index), y.reindex(test_index)
            
            #Usamos evalua_metodo para obtener el ecm y agregamos los datos al dataframe
            if nombre == "Regresión logística":
                evalua_metodo(modelo(penalty = penalty, C = 1/hiper, solver='saga', l1_ratio = 0.5,random_state=1, max_iter = 1000) , x_train , y_train, x_test, y_test)
            if nombre == "KNN":
                evalua_metodo(modelo(n_neighbors = hiper) , x_train , y_train, x_test, y_test)
            if nombre == "Arbol de decisión" or nombre  == "Boosting":
                evalua_metodo(modelo(max_depth = hiper, random_state = 1) , x_train , y_train, x_test, y_test)
            if nombre == "Support Vector Machines (SVM)":
                evalua_metodo(modelo(C = 1/hiper, random_state = 1, max_iter = -1) , x_train , y_train, x_test, y_test)
            if nombre  == "Bagging":
                evalua_metodo(modelo(n_estimators = hiper, random_state = 1) , x_train , y_train, x_test, y_test)
            if nombre == "Boosting" or nombre  == "Random Forests":
                evalua_metodo(modelo(n_estimators = hiper, random_state = 1) , x_train , y_train, x_test, y_test)
            
            ecm = mean_squared_error(Y_pred, y_test)
            datos = datos.append({"Modelo" : nombre, "Particion" : i, "Penalty" : penalty if (nombre == "Regresión logística") else "-", "Hiperparámetro": hiper ,"ECM" : ecm}, ignore_index = True)
    
    #Reseteamos el indice para arreglar un problema de formato     
    datos.reset_index(inplace = True)
    
    #Agrupamos el dataframe en base a modelo, penalty y lambda para calcular el ECM promedio entre las particiones
    datos = datos.groupby(["Modelo", 'Penalty', "Hiperparámetro"]).agg({'ECM':'mean'})

    return datos

In [5]:
def evalua_config(nombre, modelo, hyper_list, X, y, penalty = ["l1"], p = 5):
    '''
    Esta función utiliza la función cross_validation y elije el hiperparámetro óptimo (el que minimza ECM promedio en las distintas particiones) para cada modelo.
    Input:
        modelo: Modelo a utilizar.
        hyper_list: Lista de hiperparámetros a evaluar.
        X: Matriz de predictores.
        y: Variable dependiente.
        penalty: Lista con los tipos de regularización ("l1": Lasso, "l2": Ridge, "elasticnet": Elastic Net) a probar.
        p: Cantidad de particiones a realizar en cross validation.
    Output:
        Dataframe con un hiperparámetro óptimo y el ECM correspondiente para cada modelo.
    '''
    
    #Creamos el dataframe donde guardamos el output
    lambdas_opt = pd.DataFrame(columns=["Modelo", "Penalty", "Hiperparámetro", "ECM"])
    
    #Iteramos sobre los distintos penalties
    if nombre == "Regresión logística":
        for pen in penalty:
            #Usamos cross_validation para obtener los resultados en base al penalty
            datos2 = cross_validation(X, y, nombre, modelo, pen, hyper_list, p)
    else:
        datos2 = cross_validation(X, y, nombre, modelo, "-", hyper_list, p)
    datos2.reset_index(inplace = True)
    #Lo agregamos a la matriz de output
    lambdas_opt = lambdas_opt.append(datos2.loc[datos2["ECM"].idxmin()], ignore_index = True)
       
    return lambdas_opt

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.model_selection import train_test_split

def evalua_multiples_metodos(X, y, penalty = ["l1", "l2", "elasticnet"], logreg_lambda = [0.00001, 0.001, 0.1, 1, 10], k_neigh = [1, 5, 10], max_depth = [1, 3, 5], C = [0.001, 0.1, 10], n_estimators = [1, 20, 50], p = 5):
    '''
    Esta función itera sobre varios modelos (Regresión logística, Análisis discriminante lineal, KNN, Arbol de decisión, Support Vector Machines (SVM), Bagging, Random Forests, Boosting) para obtener varias métricas de performance (VP, FP, VN, FN, Accuracy, AUC, ECM) bajo distintos esquemas, y las guarda en un dataframe. Además, realiza primero validación cruzada para obtener el hiperparámetro óptimo.
    Input:
        X: Matriz de predictores.
        y: Variable dependiente.
        penalty: Lista con los tipos de regularización ("l1": Lasso, "l2": Ridge, "elasticnet": Elastic Net) a probar en cross validation
        logreg_lambda: Lista de hiperparámetros a evaluar en regresión logística.
        p: Cantidad de particiones a realizar en cross validation.
        k_neigh: Lista de valores de k a evaluar en k vecinos cercanos.
        max_depth: Lista de valores a evaluar en árboles de decisión y Boosting.
        C: Lista de valores a evaluar en SVM.
        n_estimators: Lista de valores a evaluar en Random Forest y Bagging.
    Output:
        Matriz por cada modelo con el hiperparámetro óptimo, y las métricas de performance.
    '''
    
    #Dividimos la muestra en test y train para usar en los casos que no utilizan cross_validation
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=101)
    
    #Creamos un diccionario para iterar sobre los modelos
    modelos_lista = {"Regresión logística" : LogisticRegression, "Análisis discriminante lineal" : LinearDiscriminantAnalysis, "KNN" : KNeighborsClassifier, "Arbol de decisión" : DecisionTreeClassifier, "Support Vector Machines (SVM)" : SVC, "Bagging" : BaggingClassifier, "Random Forests" : RandomForestClassifier, "Boosting" : GradientBoostingClassifier}
    
    #Creamos el dataframe de output
    output = pd.DataFrame(columns = ["Modelo", "Penalty", "Hiperparámetro", "VP", "FP", "VN", "FN", "Accuracy", "AUC", "ECM"])
    
    #Iteramos sobre los modelos
    for nombre, modelo in modelos_lista.items():
        #Si es una regresión logística, obtenemos el lambda óptimo con evalua_config y luego usamos ese lambda para obtener los resultados con evalua_metodo
        if nombre != "Análisis discriminante lineal":
            
            if nombre == "Regresión logística":
                lambdas_opt = evalua_config(nombre, modelo, logreg_lambda, X, y, penalty, p)
                output = output.append(evalua_metodo(modelo(penalty = lambdas_opt["Penalty"][0], C = 1/lambdas_opt["Hiperparámetro"][0], l1_ratio = 0.5, solver = "saga"), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre == "KNN":
                lambdas_opt = evalua_config(nombre, modelo, k_neigh, X, y, p = p)
                output = output.append(evalua_metodo(modelo(np.int_(lambdas_opt["Hiperparámetro"][0])), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre == "Arbol de decisión" or nombre  == "Boosting":
                lambdas_opt = evalua_config(nombre, modelo, max_depth, X, y, p = p)
                output = output.append(evalua_metodo(modelo(max_depth = 3, random_state = 1), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre == "Support Vector Machines (SVM)":
                lambdas_opt = evalua_config(nombre, modelo, C, X, y, p = p)
                output = output.append(evalua_metodo(modelo(C = 3, random_state = 1), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            if nombre  == "Random Forests" or nombre == "Bagging":
                lambdas_opt = evalua_config(nombre, modelo, n_estimators, X, y, p = p)
                output = output.append(evalua_metodo(modelo(n_estimators = np.int_(lambdas_opt["Hiperparámetro"][0]), random_state = 1), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)

                
            output.loc[len(output)-1, ["Modelo", "Penalty", "Hiperparámetro"]] = lambdas_opt.at[0, "Modelo"], lambdas_opt.at[0, "Penalty"], lambdas_opt.at[0, "Hiperparámetro"]
            
        #Para el resto de los casos, simplemente usamos evalua_metodo para obtener los resultados
        else:
            output = output.append(evalua_metodo(modelo(), X_train, Y_train, X_test, Y_test)[0], ignore_index = True)
            output.loc[len(output)-1, "Modelo"] = nombre
            output.fillna("-", inplace = True)
    return output

In [7]:
# Establecemos nuestras dos variables, la dependiente (y) y la independiente (x)
default = df['Default']
X = df.drop("Default", axis = 1)

In [8]:
df['Default'].value_counts()

0    275
1     20
Name: Default, dtype: int64

In [9]:
lowcorrelated_features = set()
correlation_matrix = df.corr()
correlation_matrix["Default"].sort_values(ascending=False)

Default                 1.000000
DT.DIS.DECB.CD          0.126058
DT.DOD.DEPS.CD          0.124830
DT.DOD.DPPG.CD          0.124823
DT.DOD.PUBS.CD          0.124741
                          ...   
PA.NUS.FCRF            -0.099311
FI.RES.TOTL.DT.ZS.1    -0.121378
FI.RES.TOTL.DT.ZS      -0.128018
BX.KLT.DINV.WD.GD.ZS   -0.137376
DBT-EXP                -0.155620
Name: Default, Length: 70, dtype: float64

In [10]:
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >0.0<0.05:
            colname = correlation_matrix.columns[i]
            lowcorrelated_features.add(colname)
len(lowcorrelated_features)

69

In [11]:
print(lowcorrelated_features)

{'PA.NUS.FCRF', 'DT.DOD.DECT.CD', 'GC.DOD.TOTL.GD.ZS', 'DT.DIS.PRPG.CD', 'FP.CPI.TOTL.ZG', 'NE.CON.GOVT.ZS', 'DT.DOD.ALLC.CD', 'NE.EXP.GNFS.ZS', 'DT.DIS.DLXF.CD', 'NE.IMP.GNFS.ZS', 'DT.GPA.DPPG', 'DT.TDS.DECT.CD', 'NY.GDP.PCAP.CD', 'Country', 'DT.DIS.DEGG.CD', 'DT.INR.DPPG', 'GC.REV.GOTR.ZS', 'DT.TDS.PRPG.CD', 'BN.CAB.XOKA.GD.ZS', 'DT.NFL.DEGG.CD', 'GC.TAX.TOTL.GD.ZS', 'NY.GDP.MKTP.KD.ZG', 'DT.NFL.DSTC.CD', 'BX.KLT.DINV.WD.GD.ZS', 'DT.DOD.DLXF.CD', 'DT.INT.DECT.CD', 'DT.MAT.DPPG', 'DT.NFL.DPNG.CD', 'FI.RES.TOTL.DT.ZS', 'DT.DOD.DEGG.CD', 'FI.RES.TOTL.DT.ZS.1', 'Unnamed: 69', 'DT.DIS.DEPS.CD', 'DT.GRE.DPPG', 'DT.DIS.DOPS.CD', 'NE.DAB.TOTL.ZS', 'NY.GDP.MKTP.PP.CD', 'GC.REV.XGRT.GD.ZS', 'NY.GDP.DEFL.KD.ZG.AD', 'SH.XPD.CHEX.GD.ZS', 'DT.NFL.DECT.CD', 'DT.TDS.DEPS.CD', 'DT.DOD.DPPG.CD', 'DT.NFL.DEPS.CD', 'DT.DOD.PUBS.CD', 'FI.RES.TOTL.CD', 'DT.DIS.DECB.CD', 'DT.TDS.DOPS.CD', 'DT.TDS.DECB.CD', 'NY.GDP.DEFL.KD.ZG', 'DT.DOD.PRVS.CD', 'FM.LBL.BMNY.IR.ZS', 'DT.DOD.DEPS.CD', 'DT.NFL.DLXF.CD', 'DT.N

In [12]:
df.drop(labels=lowcorrelated_features, axis=1, inplace=True)

In [13]:
evalua_multiples_metodos(X, default, ["l1", "l2"], p = 5)

/Users/ma.mercedesgarciafagalde/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/ma.mercedesgarciafagalde/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ma.mercedesgarciafagalde/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/Users/ma.mercedesgarciafagalde/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ma.mercedesgarciafagalde/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1106: UserWarning: l1_ratio 

,Modelo,Penalty,Hiperparámetro,VP,FP,VN,FN,Accuracy,AUC,ECM
0,Regresión logística,l2,0.00001,81.0,0.0,0.0,8.0,0.910112,0.500000,0.089888
1,Análisis discriminante lineal,-,-,74.0,7.0,0.0,8.0,0.831461,0.456790,0.168539
2,KNN,-,5.0,81.0,0.0,0.0,8.0,0.910112,0.500000,0.089888
3,Arbol de decisión,-,1.0,75.0,6.0,0.0,8.0,0.842697,0.462963,0.157303
4,Support Vector Machines (SVM),-,10.0,81.0,0.0,0.0,8.0,0.910112,0.500000,0.089888
5,Bagging,-,50.0,78.0,3.0,0.0,8.0,0.876404,0.481481,0.123596
6,Random Forests,-,20.0,81.0,0.0,0.0,8.0,0.910112,0.500000,0.089888
7,Boosting,-,1.0,74.0,7.0,0.0,8.0,0.831461,0.456790,0.168539


In [14]:
## Para ver desigualdad entre 1 y 0 en variable default
count_classes = pd.value_counts(df['Default'], sort = True)
count_classes.plot(kind = 'bar', rot=0)

plt.title("Frequency by observation number")
plt.xlabel("Default")
plt.ylabel("Number of Observations");

KeyError: 'Default'

In [ ]:
## Pearson correlation entre variables
import pandas as pd 
from scipy.stats import pearsonr 
  
list1 = df['DT.DOD.DSTC.ZS'] 
list2 = df['Default'] 
  
corr, _ = pearsonr(list1, list2) 
print('Pearsons correlation: %.3f' % corr) 

In [ ]:
## Matriz de correlacion entre variables 

import matplotlib.pyplot as plt
import seaborn as sns

correlation_df=df[['Default', 'GC.DOD.TOTL.GD.ZS','NY.GDP.MKTP.KD.ZG', 'DT.DOD.DSTC.ZS']]
corr = correlation_df.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right')